In [23]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn
from torch import optim
# torch.__version__


print("Helllo torch")

Helllo torch


In [8]:
torch.__version__

'1.7.1'

In [9]:
available_device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{available_device} is available")
device = torch.device(device=available_device)
device

cuda is available


device(type='cuda')

# CNN in pytorch

- Stanfords notes Convolutional neural network
     - https://cs231n.github.io/convolutional-networks/
  
  
- Conv2d document from pytorch
    - https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d
    - In the simplest case, the output value of the conv layer with input size $(N, C_{\text{in}}, H, W)$ has an output size $(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})$ 
    - N is a batch size, C denotes a number of channels, H is a height of input planes in pixels, and W is width in pixels.
    
    
- Pooling document from pytorch
    - https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d

### Convolution Layer Pytorch Documentation

$$ Input: (N, C_{in}, H_{in}, W_{in}) $$

$$ Output: (N, C_{out}, H_{out}, W_{out}) $$

$$where,$$

$$H_{out} = \left\lfloor\frac{H_{in} + 2 \times \text{padding}[0] - \text{dilation}[0] \times (\text{kernel_size}[0] - 1) - 1}{\text{stride}[0]} + 1\right\rfloor$$

$$W_{out} = \left\lfloor\frac{W_{in} + 2 \times \text{padding}[1] - \text{dilation}[1] \times (\text{kernel_size}[1] - 1) - 1}{\text{stride}[1]} + 1\right\rfloor$$



In [3]:
# With square kernels and equal stride
conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5 ) #,stride=2, padding=1)
input = torch.randn(64, 3, 32, 32)
c1_output = conv1(input)
c1_output.shape

torch.Size([64, 6, 28, 28])

#### The parameters kernel_size, stride, padding, dilation can either be:
- a single int – in which case the same value is used for the height and width dimension
- a tuple of two ints – in which case, the first int is used for the height dimension, and the second int for the width dimension



In [69]:
# With square kernels and equal stride
c2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5))  # ,stride=(2,2),padding=1)

# non-square kernels and unequal stride and with padding
# m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
max_output = m_pool(c1_output)

c2_output = c2(max_output)
c2_output.shape

torch.Size([64, 16, 10, 10])

### Max Pooling layer Pytorch Documentation

In [70]:
# pool of square window of size=3, stride=2
m_pool = nn.MaxPool2d(2, stride=2)
# pool of non-square window
# m = nn.MaxPool2d((3, 2), stride=(2, 1))

# input = torch.randn(20, 16, 50, 32)
# this output comes from the above conv layer
max_output_2 = m_pool(c2_output)
max_output_2.shape

torch.Size([64, 16, 5, 5])

In [48]:
max_linear = max_output.reshape(max_output.shape[0], -1)
max_linear.shape

torch.Size([20, 784])


## Finally implementation on CIFAR10 Dataset

### Loading Dataset

In [3]:
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
batch_size = 64

In [4]:
train_set = datasets.CIFAR10(root="pytorch_dataset",
                           download=True,
                           transform=transforms.ToTensor()
                          )
test_set = datasets.CIFAR10(root="pytorch_dataset",
                           train=False,
                           download=True,
                           transform=transforms.ToTensor()
                          )
print("done")

Files already downloaded and verified
Files already downloaded and verified
done


In [5]:
train_loader = DataLoader(dataset=train_set, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(dataset=test_set, shuffle=True, batch_size=batch_size)

In [123]:
for x, y in train_loader:
    print(x.shape)
    print((x/255).mean())
    break
    
# 32*

torch.Size([64, 3, 32, 32])
tensor(0.0019)


In [62]:
32/4

8.0

### Creating the model

In [6]:
import torch.nn.functional as F

In [7]:
class CNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5,128)
        self.fc2 = nn.Linear(128,num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(x.shape[0], -1) # flatten CONV -> FC
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x    

### Parameter Initialized

In [8]:
n_channels = 3
num_classes = 10
num_epochs = 3
lr = 0.001

### Model Initialized

In [9]:
# cnn model initialized using the CNN class defined earlier
cnn_model = CNN(n_channels, num_classes).to(device=device)

### Loss define

In [10]:
criterion = nn.CrossEntropyLoss()

### Optimizer Define

In [12]:
# from torch.optim import Adam
from torch import optim
optimizer = optim.Adam(cnn_model.parameters(), lr=lr)

### Training the cnn model

In [16]:
for epoch in range(num_epochs):
    for x_train, y_train in train_loader:
        x_train = x_train.to(device)
        y_train = y_train.to(device)
#         x_train = x_train/255
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        y_pred = cnn_model(x_train)
        
        loss = criterion(y_pred, y_train)
        
        loss.backward()
        
        optimizer.step()

    print(f" loss at the end of {epoch} epoch is {loss}")

print("Done Training")

 loss at the end of 0 epoch is 1.8232377767562866
 loss at the end of 1 epoch is 1.0600694417953491
 loss at the end of 2 epoch is 1.1026233434677124
Done Training


In [135]:
def accuracy(test_loader, model,cuda=True):
    n_correct = 0
    n_total = 0
    with torch.no_grad():
        for x_test, y_test in test_loader:
            if cuda:
                x_test = x_test.to(device)
                y_test = y_test.to(device=device)
            x_test = x_test.float()/255
            preds = model(x_test)
            _, preds_idx = preds.max(1)
            n_correct += (preds_idx==y_test).sum().item()
            n_total += x_test.size(0)
    print(f"Out of {n_total} images {n_correct} where correctly classified")
    acc = (n_correct/n_total) * 100
    print(f"Accuracy of the model is {acc:.2f}")            

In [18]:
accuracy(test_loader, cnn_model)

Out of 10000 images 5333 where correctly classified
Accuracy of the model is 53.33


## Saving the model

In [19]:
checkpoint = {'state_dict' : cnn_model.state_dict(), 'optimizer' : optimizer.state_dict()}
# save


In [20]:
PATH = './cifar_net.pth.tar'
torch.save(checkpoint, PATH)
print("check point saved")

## Loading the model

In [24]:
net = CNN()
l_optimizer = Adam(net.parameters(), lr=lr)
checkpoint_loaded = torch.load(PATH)

net.load_state_dict(checkpoint['state_dict'])
l_optimizer.load_state_dict(checkpoint['optimizer'])

In [25]:
accuracy(test_loader, net,cuda=False)

Out of 10000 images 5333 where correctly classified
Accuracy of the model is 53.33


## TODO

- [ ] Create a predicting functions
- [ ] Print loss after certain batch size
- [ ] Look into the data with high loss in training as shown in fastai course

# CNN on Custom Dataset

In [2]:
import os
import pandas as pd
import torch 
from torch.utils.data import Dataset
from skimage import io
from skimage.transform import resize

In [14]:
folder_path = "../NLPR/myLPR/finalFolder/"
folders = os.listdir(folder_path)
folders

['9', '5', '3', '4', '1', '7', 'pa', '8', '6', '2', '0', 'BA']

In [62]:
len(folders)

12

In [11]:
# license_csv = pd.read_csv("finalPathAndImage.csv",low_memory=False)
# len(license_csv)

16980

In [46]:
# digit_image_label  = pd.DataFrame()
# all_images = []
# all_labels = []
# for digit_folder in folders:
#     digit_path = folder_path + digit_folder
# #     print(digit_path)
#     images = os.listdir(digit_path)
# #     print(images[:5])
# #     print(final['label'][:5])
#     all_images.extend(images)
#     all_labels.extend(len(images)*[digit_folder])

# print(len(all_images))
# #     break
# digit_image_label['image_path'] = pd.Series(all_images)
# digit_image_label['label'] = pd.Series(all_labels)

17315


In [47]:
# len(digit_image_label)

17315

In [51]:
# digit_image_label.head()

In [49]:
# digit_image_label.to_csv("digit_and_label.csv", index=False)

In [13]:
digit_label = "digit_and_label.csv"
root_dir = "../NLPR/myLPR/finalFolder/"
batch_size = 64

In [4]:
class NepaliDigitDataset(Dataset):
    def __init__(self,csv_file, root_dir, transform=None):
        self.root = root_dir
        self.transform = transform
        self.digit_frame =  pd.read_csv(csv_file, low_memory=False)
    def __len__(self):
        return len(self.digit_frame)
    def __getitem__(self, index):
        digit = self.digit_frame.iloc[index, 1]
        img_name = self.digit_frame.iloc[index, 0]
        img_path = os.path.join(self.root + digit + "/" + img_name)
        image = io.imread(img_path)
        image = resize(image,(32, 32,3))
        if digit=="pa":
            digit=10
        if digit=="BA":
            digit=11
        y_label = torch.tensor(int(digit))
        if self.transform:
            image = self.transform(image)
            
        return (image, y_label)

### Load Data

In [28]:
transform = transforms.Compose([
            transforms.ToTensor(),
])
dataset = NepaliDigitDataset(digit_label,root_dir,transform=transform)

In [30]:
train_set, test_set = torch.utils.data.random_split(dataset,[2315,15000])

In [31]:
train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_set, shuffle=True, batch_size=batch_size)

In [32]:
for x,y in train_loader:
    print(x.shape)
    print(y)
    break

torch.Size([64, 3, 32, 32])
tensor([ 2,  0, 11,  4,  1,  2,  1,  9, 11,  6,  3,  9,  3,  0,  3,  8,  6,  6,
         0,  0,  9,  0, 11,  5, 11, 11,  8,  0, 10,  5,  1,  8,  6, 10,  6,  3,
         6,  0,  4,  8,  0, 10,  5,  0,  2, 11,  2,  7,  2,  9,  6,  1, 11,  6,
         5,  3,  0, 11,  6,  0,  9, 11,  7,  3])


## Creating model

In [33]:
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5,128)
        self.fc2 = nn.Linear(128,num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(x.shape[0], -1) # flatten CONV -> FC
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x    

## parameter initialized

In [38]:
n_channels = 3
num_classes = 12
num_epochs = 20
lr = 0.001

# n_channels = 3
# num_classes = 10
# num_epochs = 3
# lr = 0.001

## model initialized

In [35]:
# cnn model initialized using the CNN class defined earlier
cnn_model = CNN(n_channels, num_classes).to(device=device)

## Loss and optimizer

In [39]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=lr)

## Learning Rate Scheduler

In [40]:
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True) # factor=0.1

## Training the model

In [41]:
for epoch in range(num_epochs):
    losses = []
    for x_train, y_train in train_loader:
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        x_train = x_train.float()/255
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        y_pred = cnn_model(x_train)
        
        loss = criterion(y_pred, y_train)
        
        losses.append(loss.item())
        
        loss.backward()
        
        optimizer.step()
    mean_loss = sum(losses)/ len(losses)
    scheduler.step(mean_loss)
    
    print(f" loss at the end of {epoch} epoch is {loss}")

print("Done Training")

 loss at the end of 0 epoch is 2.3432869911193848
 loss at the end of 1 epoch is 2.623555898666382
 loss at the end of 2 epoch is 2.3396313190460205
 loss at the end of 3 epoch is 2.568519353866577
 loss at the end of 4 epoch is 2.4552524089813232
 loss at the end of 5 epoch is 2.457399845123291
 loss at the end of 6 epoch is 2.6387178897857666
 loss at the end of 7 epoch is 2.519538164138794
 loss at the end of 8 epoch is 2.285705327987671
 loss at the end of 9 epoch is 2.296299457550049
 loss at the end of 10 epoch is 2.18076753616333
 loss at the end of 11 epoch is 2.6413867473602295
 loss at the end of 12 epoch is 2.3331358432769775
 loss at the end of 13 epoch is 2.306072235107422
 loss at the end of 14 epoch is 2.535757303237915
 loss at the end of 15 epoch is 2.498894453048706
 loss at the end of 16 epoch is 2.5984904766082764
 loss at the end of 17 epoch is 2.4779984951019287
 loss at the end of 18 epoch is 2.0723512172698975
 loss at the end of 19 epoch is 2.549471616744995
Do

In [136]:
accuracy(test_loader, cnn_model)

Out of 6000 images 1308 where correctly classified
Accuracy of the model is 21.80


In [42]:
torch.cuda.empty_cache()